<center><b><h1>  Where were revolutions located between 1840 and 1860, based on article titles from Dutch and English newspapers?</b></center>



---------------------------------------------------------------------------------------------------------------------------
The Dutch dataset was constructed querying Delpher on the keyword 'revolutie' for the period 1840-1860, with OCR confidence between 80-100 per cent. Since Delpher has no bulk download option, we've instead used [I-Analyzer](https://ianalyzer.hum.uu.nl) (developed by the [DH Lab at Utrecht University](https://dig.hum.uu.nl/)). This tool allows to query and download Delpher search results for this period.

The same method has been used to fetch English articles from The Times, for the same period. Only addition is to exclude articles mentioning "industrial", since "industrial revolution" was also an often used term this period.

To plot the locations mentioned in the article titles several steps are needed: Named Entity Recognition, extracting the locations, geocoding these, and obtaining a map to plot them. Here we've used (mainly) R for this, but Python would work as well. 

Apart from having R installed, you'll also need a python executable with [spaCy](https://spacy.io/usage) installed, and the corresponding [Dutch language model](https://spacy.io/models/nl#nl_core_news_sm). If you opt for a Google Map you'll also need a [Google API key for its Maps platform](https://developers.google.com/maps/faq). This is free as long as you don't exceed $200 in API calls a month (which usually does't happen).

---------------------------------------------------------------------------------------------------------------------------

First we load the required libraries

In [100]:
library(data.table)
library(spacyr)
library(maps)
library(rasterVis)
library(raster)
library(ggmap)
library(ggplot2)
library(ggpubr)



Attaching package: 'ggpubr'


The following object is masked from 'package:raster':

    rotate




Load the Delpher dataset (currently locally stored)

In [13]:
revol <- fread("C:\\Users\\Schal107\\Documents\\UBU\\Team DH\\Delpher\\dutchnewspapers-public_query=revolutie_date=1840-01-01 1860-12-31_ocr=80 100_sort=date,desc.csv")
setDT(revol)

Inspect one row of the data. You'll see that it's long because the content of the article is still included. We will not work with that now, since many article_titles already mention a placename. In a next step we could include article content, of course, and use the same workflow as below.

In [14]:
revol[5814]

date       issue_number category circulation      temporal
1 1848-10-23 55           artikel  Regionaal/lokaal Middag  
  article_title                               
1 UTRECHT, 21 October. DE REVOLUTIE TE WEENEN.
  content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [51]:
colnames(revol)

[1] "date"            "issue_number"    "category"        "circulation"    
 [5] "temporal"        "article_title"   "content"         "url"            
 [9] "newspaper_title" "ocr"

Let's keep only the variables we want to continue to work with. And inspect the first rows of this subset. You'll already notice the mentioning of "FRANKRIJK" (France) and "WEENEN" (Vienna) here.

In [15]:
x <- revol[,c("date", "article_title", "url")]

In [16]:
x[5851:5858]

date,article_title,url
<date>,<chr>,<chr>
1848-10-13,REVOLUTIE IN WEENEN OP DEN 6 EN 7 OCTOBER.,http://resolver.kb.nl/resolve?urn=ddd:010772343:mpeg21:a0051
1848-10-13,ZWITSERLAND. BERN 8 Oct.,http://resolver.kb.nl/resolve?urn=ddd:010779162:mpeg21:a0002
1848-10-13,"Brussel , 11 October.",http://resolver.kb.nl/resolve?urn=ddd:010067138:mpeg21:a0009
1848-10-12,"FRANKRIJK. PARIJS, den 8sten October.",http://resolver.kb.nl/resolve?urn=ddd:010174429:mpeg21:a0001
1848-10-12,Een blik op FrankrijKs consti- tuerende vergadering.,http://resolver.kb.nl/resolve?urn=ddd:010151491:mpeg21:a0001
1848-10-10,LONDEN. 7 October ('savonds).,http://resolver.kb.nl/resolve?urn=ddd:010519531:mpeg21:a0007
1848-10-10,"FRANKRIJK. PARIJS, 3 October.",http://resolver.kb.nl/resolve?urn=ddd:010772342:mpeg21:a0017
1848-10-10,DUITSCHLAND.,http://resolver.kb.nl/resolve?urn=ddd:010929387:mpeg21:a0008


Not required for now, but below demonstrates how to extract years from the date variable.

In [17]:
x$date <- as.Date(x$date)
x[, year := as.numeric(substr(x$date, 1,4))]

In [18]:
setDT(x)
x[, .N, list(year)][order(-year)][1:10]

year,N
<dbl>,<int>
1860,1006
1859,806
1858,290
1857,321
1856,471
1855,217
1854,317
1853,328
1852,290


For better Named Entity Recognition we will change the article_title variable to lower case strings. 
SpaCy has the tendency to recognize uppercase strings as corporations.

In [19]:
x$article_title <- tolower(x$article_title)

Now we need to call in action the [Dutch language model](https://spacy.io/models/nl/#nl_core_news_sm) from spaCy to perform NER. If you want to replicate this analysis, you need to have this installed on your local machine. Read the website for installation instructions. 

In [20]:
spacy_initialize(model = "nl_core_news_sm")

spaCy is already initialized



NULL

Below we parse the article_title to SpaCy and perform NER. You'll see that it has recognized at least some locations in the resulting table.

In [21]:
parsedtxt <- spacy_parse(x$article_title, lemma = FALSE, entity = TRUE, nounphrase = TRUE)

In [22]:
locations <- entity_extract(parsedtxt)

In [23]:
setDT(locations)
top100 <- locations[entity_type == "GPE", .N, list(entity) ][order(-N)]

In [24]:
head(top100)

entity,N
<chr>,<int>
amsterdam,652
parijs,246
buitenland,212
londen,77
rome,35
madrid,33


Before we can plot them on a map, we need to add coordinates to the placenames. We'll use the Google Maps API for that. Notice that you will need your own key to do so. You can register for a free API key at Google. *(It seems impossible to hide the API call output, so just ignore that and scroll down.)*

In [5]:
google_key <- fread("C:\\Users\\Schal107\\Documents\\UBU\\Team DH\\Delpher\\google_key.txt")
register_google(key = paste0(google_key$key))

In [25]:
coordinates <- geocode(top100$entity)

Source : https://maps.googleapis.com/maps/api/geocode/json?address=amsterdam&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=parijs&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=buitenland&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=londen&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=rome&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=madrid&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=china&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=portugal&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=ingezonden&key=xxx

Warning message:
"Geocoding "ingezonden" failed with error:

"
Source : https://maps.googleapis.com/maps/api/geocode/json?address=brussel&key=xxx

"brussel" not uniquely geocoded, using "brussels, belgium"

Source : https://maps.googleapis.com

In [26]:
head(coordinates)

lon,lat
<dbl>,<dbl>
4.9041389,52.36757
2.3522219,48.85661
4.2314198,51.11525
-0.1275862,51.50722
12.4963655,41.90278
-3.7037902,40.41678


After the geocoding we merge the retrieved coordinates with the placenames.

In [28]:
coordinates_delpher <- cbind(top100, coordinates)
head(coordinates_delpher)

entity,N,entity,N,lon,lat
<chr>,<int>,<chr>,<int>,<dbl>,<dbl>
amsterdam,652,amsterdam,652,4.9041389,52.36757
parijs,246,parijs,246,2.3522219,48.85661
buitenland,212,buitenland,212,4.2314198,51.11525
londen,77,londen,77,-0.1275862,51.50722
rome,35,rome,35,12.4963655,41.90278
madrid,33,madrid,33,-3.7037902,40.41678


<center><b><h1>  Now we'll do the same for articles from The Times</b></center>

In [3]:
times <- fread("C:\\Users\\Schal107\\Documents\\UBU\\Team DH\\Delpher\\times_query=revolution_-_industrial_date=1840-01-01 1860-12-31_ocr=80 100.csv")
colnames(times)

[1] "edition"  "issue"    "volume"   "date-pub" "content"  "title"    "ocr"     
[8] "id"

Now we'll perform NER on the English titles.

In [4]:
spacy_initialize(model = "en_core_web_sm")

Finding a python executable with spaCy installed...

spaCy (language model: en_core_web_sm) is installed in C:\Users\Schal107\AppData\Local\Programs\Python\Python39\python.exe

successfully initialized (spaCy Version: 3.3.0, language model: en_core_web_sm)

(python options: type = "python_executable", value = "C:\Users\Schal107\AppData\Local\Programs\Python\Python39\python.exe")



In [6]:
parsedtxt2 <- spacy_parse(times$title, lemma = FALSE, entity = TRUE, nounphrase = TRUE)

In [8]:
locations2 <- entity_extract(parsedtxt2)

In [10]:
setDT(locations2)
times_locations <- locations2[entity_type == "GPE", .N, list(entity) ][order(-N)]
head(times_locations)

entity,N
<chr>,<int>
London,781
France,305
Paris,244
Austria,199
Prussia,122
Spain,118


In [11]:
coordinates_times <- geocode(times_locations$entity)

Source : https://maps.googleapis.com/maps/api/geocode/json?address=London&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=France&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Paris&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Austria&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Prussia&key=xxx

Warning message:
"Geocoding "Prussia" failed with error:

"
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Spain&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Ireland&key=xxx

"Ireland" not uniquely geocoded, using "ireland"

Source : https://maps.googleapis.com/maps/api/geocode/json?address=America&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Portugal&key=xxx

Source : https://maps.googleapis.com/maps/api/geocode/json?address=The_French_Republic&key=xxx

Source : https://maps.googleapis.com/maps/a

In [12]:
coordinates_times <- cbind(times_locations, coordinates_times)
head(coordinates_times)

entity,N,lon,lat
<chr>,<int>,<dbl>,<dbl>
London,781,-0.1275862,51.50722
France,305,2.2137490,46.22764
Paris,244,2.3522219,48.85661
Austria,199,14.5500720,47.51623
Prussia,122,NA,NA
Spain,118,-3.7492200,40.46367


To identify the two datasets after merging, we add a variable and rename "N".

In [34]:
coordinates_times[, dataset := "times",]
coordinates_delpher[, dataset := "delpher"]
setnames(coordinates_times, "N", "n_times")
setnames(coordinates_delpher, "N", "n_delpher")

entity,n_times,lon,lat,dataset
<chr>,<int>,<dbl>,<dbl>,<chr>
London,781,-0.1275862,51.50722,times
France,305,2.2137490,46.22764,times
Paris,244,2.3522219,48.85661,times
Austria,199,14.5500720,47.51623,times
Prussia,122,NA,NA,times
Spain,118,-3.7492200,40.46367,times


entity,n_delpher,entity,N,lon,lat,dataset
<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>
amsterdam,652,amsterdam,652,4.9041389,52.36757,delpher
parijs,246,parijs,246,2.3522219,48.85661,delpher
buitenland,212,buitenland,212,4.2314198,51.11525,delpher
londen,77,londen,77,-0.1275862,51.50722,delpher
rome,35,rome,35,12.4963655,41.90278,delpher
madrid,33,madrid,33,-3.7037902,40.41678,delpher


In [69]:
setnames(coordinates_times, "entity", "entity_times")
setnames(coordinates_delpher, "entity", "entity_delpher")

coordinates_times <- coordinates_times[!is.na(lon),]
coordinates_delpher <- coordinates_delpher[!is.na(lon),]

lat,entity_times,n_times,lon.x,dataset.x,entity_delpher,n_delpher,lon.y,dataset.y
<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,<dbl>,<chr>
52.13263,Holland,2,5.291266,times,holland,2,5.291266,delpher
52.13263,Netherlands,1,5.291266,times,nederland,3,5.291266,delpher
52.13263,Netherlands,1,5.291266,times,holland,2,5.291266,delpher
52.35552,England,17,-1.174320,times,engeland,1,-1.174320,delpher
52.35552,The_English_Reformation,1,-1.174320,times,engeland,1,-1.174320,delpher
52.52001,Berlin,4,13.404954,times,berlijn,1,13.404954,delpher


<center><b><h1>  Next: plot coordinates from Delpher and Times on a map</b></center>

Now we'll need a map! You can use the standard Google Map and define a centre using latitute and longitude. However, for historical data I like to remove roads and names from the map. You can do that by making your own map at [Google Mapstyle](https://mapstyle.withgoogle.com/). With the same API key as you've use for the geocoding, you can export the URL of your map of choice, and paste it below in the function `get_googlemap`. For some reason, though, you first need to extract the lat and long from this URL, and the zoom level (see below). Then you paste the remaining URL from the first mention of '&maptype' behind `path =` (don't forget to include it in quotes). Then it should work!

In [70]:
europe5 <- get_googlemap(center=c(lon=10.95931966568949, lat=48.561877580811775), zoom = 4, path = "&maptype=roadmap&style=element:geometry%7Ccolor:0xf5f5f5&style=element:labels%7Cvisibility:off&style=element:labels.icon%7Cvisibility:off&style=element:labels.text.fill%7Ccolor:0x616161&style=element:labels.text.stroke%7Ccolor:0xf5f5f5&style=feature:administrative.land_parcel%7Cvisibility:off&style=feature:administrative.land_parcel%7Celement:labels.text.fill%7Ccolor:0xbdbdbd&style=feature:administrative.neighborhood%7Cvisibility:off&style=feature:landscape.natural.terrain%7Ccolor:0xffffff%7Cvisibility:on%7Cweight:4&style=feature:landscape.natural.terrain%7Celement:geometry.fill%7Cvisibility:on%7Cweight:4&style=feature:landscape.natural.terrain%7Celement:geometry.stroke%7Cvisibility:on&style=feature:poi%7Celement:geometry%7Ccolor:0xeeeeee&style=feature:poi%7Celement:labels.text.fill%7Ccolor:0x757575&style=feature:poi.park%7Celement:geometry%7Ccolor:0xe5e5e5&style=feature:poi.park%7Celement:labels.text.fill%7Ccolor:0x9e9e9e&style=feature:road%7Cvisibility:off&style=feature:road%7Celement:geometry%7Ccolor:0xffffff&style=feature:road.arterial%7Celement:labels.text.fill%7Ccolor:0x757575&style=feature:road.highway%7Celement:geometry%7Ccolor:0xdadada&style=feature:road.highway%7Celement:labels.text.fill%7Ccolor:0x616161&style=feature:road.local%7Celement:labels.text.fill%7Ccolor:0x9e9e9e&style=feature:transit.line%7Celement:geometry%7Ccolor:0xe5e5e5&style=feature:transit.station%7Celement:geometry%7Ccolor:0xeeeeee&style=feature:water%7Celement:geometry%7Ccolor:0xc9c9c9&style=feature:water%7Celement:labels.text.fill%7Ccolor:0x9e9e9e&size=480x360")


Source : https://maps.googleapis.com/maps/api/staticmap?center=48.561878,10.95932&zoom=4&size=640x640&scale=2&maptype=terrain&path=&maptype=roadmap&style=element:geometry%7Ccolor:0xf5f5f5&style=element:labels%7Cvisibility:off&style=element:labels.icon%7Cvisibility:off&style=element:labels.text.fill%7Ccolor:0x616161&style=element:labels.text.stroke%7Ccolor:0xf5f5f5&style=feature:administrative.land_parcel%7Cvisibility:off&style=feature:administrative.land_parcel%7Celement:labels.text.fill%7Ccolor:0xbdbdbd&style=feature:administrative.neighborhood%7Cvisibility:off&style=feature:landscape.natural.terrain%7Ccolor:0xffffff%7Cvisibility:on%7Cweight:4&style=feature:landscape.natural.terrain%7Celement:geometry.fill%7Cvisibility:on%7Cweight:4&style=feature:landscape.natural.terrain%7Celement:geometry.stroke%7Cvisibility:on&style=feature:poi%7Celement:geometry%7Ccolor:0xeeeeee&style=feature:poi%7Celement:labels.text.fill%7Ccolor:0x757575&style=feature:poi.park%7Celement:geometry%7Ccolor:0xe5e5e5

Now that we have our map loaded, we parse it to ggmap. Then we can add our data to it. The size of the dots corresponds to the number of times the geocoded placename is mentioned in our article titles. You'll see that Paris and France are dominant, but also that we spot some unexpected places in Italy and even Eastern Europe.

In [103]:
p <- ggmap(europe5)
times <- p + geom_point(data = coordinates_times, aes(x=lon, y=lat, size=(n_times)), shape=16, color = "red")
invisible(ggplot_build(times))
ggsave("times_map.png")

p <- ggmap(europe5)
delpher <- p + geom_point(data = coordinates_delpher, aes(x=lon, y=lat, size=(n_delpher)), shape=16, color = "blue")
invisible(ggplot_build(delpher))
ggsave("delpher_map.png")

invisible(ggarrange(times, delpher))
ggsave("all_map.png")

Saving 6.67 x 6.67 in image

Warning message:
"Removed 51 rows containing missing values (geom_point)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 13 rows containing missing values (geom_point)."
Warning message:
"Removed 51 rows containing missing values (geom_point)."
Warning message:
"Removed 13 rows containing missing values (geom_point)."
Saving 6.67 x 6.67 in image



---------------------------------------------------------------------------------------------------------------------------
<center><b><h3>  And here we have our results:</b></center>

[<img src="all_map.png" width="1850"/>](all_map.png)

*For questions and comments email <r.schalk@uu.nl>